<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/Mediator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mediator / Посредник
Определяет объект, инкапсулирующий способ взаимодействия множества объектов. Посредник обеспечивает слабую связанность системы, избавляя объекты от необходимости явно ссылаться друг на друга и позволяя тем самым независимо изменять взаимодействия между ними.
Используется когда:
- имеются объекты, связи между которыми сложны и  четко определены. Получающиеся при этом взаимозависимости не структурированы и трудны для понимания.
- нельзя повторно использовать объект, поскольку он обменивается информацией со многими другими объектами.
- поведение, распределенное между несколькими классами, должно поддаваться настройке без порождения множества подклассов.

## Пример реализации 
<img src='http://feana.ru/wp-content/uploads/2023/05/mediator.png'>

In [9]:
# общий класс диалогового окна, включающий в себя виджеты
class DialogDirector:
    def __init__(self):
        self._widgets = {}
    def widget_changed(self,widget):
        pass
    def create_widgets(self):
        pass
    def show(self):
        print('\n\n---------------------------------')
        for i in self._widgets.values():
            i.show()
        print('---------------------------------\n\n')

# общий класс виджета
class Widget:
    def __init__(self, director:DialogDirector):
        self._director = director

    def changed(self):
        self._director.widget_changed(self)
    def handle_event(self,event):
        pass
    def show(self):
        pass

# класс конкретного виджета - Списка
class ListBox(Widget):
    def set_items(self,items:list):
        self._items = items
        self.selected_index = None

    def show(self):
        print('-'*25)
        for i,value in enumerate(self._items):
            if i == self.selected_index:
                value = value.upper()
            print(value, end=' | ')
        print('\n'+'-'*25)

    def handle_event(self,event):
        self.selected_index  = self._items.index(event)
        self.changed()

    def get_selected_text(self):
        if self.selected_index!=None:
            return self._items[self.selected_index]
        else:
            return None

# класс конкретного виджета - Текстового поля
class TextField(Widget):
    def set_text(self,text):
        self.text = text

    def show(self):
        print('====>'+self.text+'<=====')

# класс конкретного виджета - Кнопки
class Button(Widget):
    def set_text(self,text):
        self.text = text
    def set_active(self):
        self.text = self.text.upper()
    def set_inactive(self):
        self.text = self.text.capitalize()
    def show(self):
        print('-'*10)
        print('|'+self.text.center(8,' ')+'|')
        print('-'*10)
    def handle_event(self,event):
        if event==True:
            self.set_active()
        else:
            self.set_inactive()
        self.changed()

# класс конкретного диалогового окна - 
# в него входят виджеты: Список, Текстовое поле, Кнопка.
# Класс отслеживает события, случающиеся с его виджетами и их коммуникацию.
class MyDialog(DialogDirector):
    def create_widgets(self,listbox:ListBox,textfield:TextField,button:Button):
        self._widgets['listbox'] = listbox
        self._widgets['textfield'] = textfield
        self._widgets['button'] = button

    def widget_changed(self,widget:Widget):
        if widget == self._widgets['listbox']:
            print('Был выбран пункт из списка!')
            self._widgets['textfield'].set_text(widget.get_selected_text())
        elif widget == self._widgets['button']:
             print('Была нажата кнопка!')
        self.show()

# Создадим диалоговое окно
d = MyDialog()
# Создадим список
lb = ListBox(d)
lb.set_items(['first', 'second', 'third'])
# Создадим текстовое поле
tf = TextField(d)
tf.set_text('sample')
# Создадим кнопку
b = Button(d)
b.set_text('Okey')
# Добавим виджеты в диалоговое окно
d.create_widgets(lb,tf,b)
# покажем получившееся диалоговое окно
print('Начальное состояние диалогового окна: \n')
d.show()
print("""Обработаем событие выбора пункта second в списке. 
В текстовом поле появится содержимое этого пункта. \n""")
lb.handle_event('second')
print('Обработаем событие нажатия кнопки\n')
b.handle_event(True)
print('Обработаем событие отжатия кнопки\n')
b.handle_event(False)

Начальное состояние диалогового окна: 



---------------------------------
-------------------------
first | second | third | 
-------------------------
====>sample<=====
----------
|  Okey  |
----------
---------------------------------


Обработаем событие выбора пункта second в списке. 
В текстовом поле появится содержимое этого пункта. 

Был выбран пункт из списка!


---------------------------------
-------------------------
first | SECOND | third | 
-------------------------
====>second<=====
----------
|  Okey  |
----------
---------------------------------


Обработаем событие нажатия кнопки

Была нажата кнопка!


---------------------------------
-------------------------
first | SECOND | third | 
-------------------------
====>second<=====
----------
|  OKEY  |
----------
---------------------------------


Обработаем событие отжатия кнопки

Была нажата кнопка!


---------------------------------
-------------------------
first | SECOND | third | 
-------------------------
=